<a href="https://colab.research.google.com/github/kavyajeetbora/nlp_doc/blob/master/notebooks/02_simple_RAG_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/kavyajeetbora/nlp_doc/raw/master/data/doc_embeddings.parquet -O "doc_embeddings.parquet"

--2024-04-13 10:02:11--  https://github.com/kavyajeetbora/nlp_doc/raw/master/data/doc_embeddings.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kavyajeetbora/nlp_doc/master/data/doc_embeddings.parquet [following]
--2024-04-13 10:02:11--  https://raw.githubusercontent.com/kavyajeetbora/nlp_doc/master/data/doc_embeddings.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3922167 (3.7M) [application/octet-stream]
Saving to: ‘doc_embeddings.parquet’

doc_embeddings.parq 100%[===================>]   3.74M  --.-KB/s    in 0.07s   

2024-04-13 10:02:12 (52.5 MB/s) - ‘doc_embeddings.parque

In [7]:
!pip install -U -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.4 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np

df = pd.read_parquet('doc_embeddings.parquet')
df.sample(5)

,page_number,sentence_chunk,chunk_char_count,chunk word count,chunk_token_count,embedding
1523,1070,"American Heart Journal. 156(1), 13-22. 20. Ols...",222,32,55.50,"[0.030198118, 0.0217555, -0.037539102, 0.03403..."
1351,951,Image by Allison Calabrese / CC BY 4.0 Physica...,1248,213,312.00,"[-3.8579645e-05, 0.05074084, -0.03621196, 0.03..."
522,346,"Fats and Cholesterol: Out with the Bad, In wit...",244,31,61.00,"[0.012333913, -0.032216914, 0.052167647, 0.087..."
22,-17,Preface UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SC...,998,158,249.50,"[0.014127462, 0.05853739, -0.020736571, 0.1036..."
675,459,Learning Activities Technology Note: The secon...,729,104,182.25,"[-0.0022831433, -0.0053499825, 0.013110881, 0...."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   page_number        1680 non-null   int64  
 1   sentence_chunk     1680 non-null   object 
 2   chunk_char_count   1680 non-null   int64  
 3   chunk word count   1680 non-null   int64  
 4   chunk_token_count  1680 non-null   float64
 5   embedding          1680 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 78.9+ KB


Since the embeddings are stored as string in the csv file, we need to convert it back to numpy arrays so that we can further use it for processing

In [6]:
df['embedding'].iloc[0].shape

(384,)

Convert the embeddings to tensor object

In [41]:
a = np.stack(df['embedding'].to_list(), axis=0)
embedding  = torch.tensor(a)
print(f"Data type of the embedding array: {embedding.dtype}")
embedding

Data type of the embedding array: torch.float32


tensor([[-1.9507e-02, -1.8256e-02,  7.1660e-04,  ..., -3.9904e-02,
         -2.1543e-02, -4.3294e-02],
        [ 2.6605e-02,  1.8785e-03, -7.0033e-03,  ..., -2.1591e-02,
         -2.1828e-02, -6.9987e-02],
        [ 3.4971e-02,  4.0597e-02, -1.9747e-02,  ..., -3.5225e-02,
         -3.1397e-02, -4.6613e-02],
        ...,
        [-3.2258e-02, -2.4768e-02, -1.1571e-02,  ..., -1.3912e-05,
          3.9905e-02, -5.2110e-02],
        [-8.8627e-02, -6.6496e-02, -1.0430e-01,  ..., -3.0128e-02,
          8.4004e-02, -4.2208e-02],
        [-1.1203e-01, -4.9535e-02, -2.4897e-02,  ...,  1.2983e-02,
          4.6920e-02, -2.8276e-02]])

## Creating a semantic search engine


Here we will build a semantic search engine, given a query for example "Macronutrients" it will semantically search and return relevant passage

In [42]:
from sentence_transformers import SentenceTransformer, util
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [10]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
## Encode the query with the same embedding model
query = "Macronutrients"
query_encode = embedding_model.encode(query, convert_to_tensor=True)

## Get similarity scores
sim_score = util.dot_score(a=query_encode, b=embedding)

In [62]:
vals, indices = torch.topk(sim_score, k=5)

for i,v in zip(indices.flatten(),vals.flatten()):
    i,v = i.numpy(), v.numpy()
    print(f"Page number: {df.loc[i,'page_number']}")
    print(f"Sentence: {df.loc[i, 'sentence_chunk']}")
    print(f'Similarity score: {v*100:.2f} %')
    print("-"*15)

Page number: 5
Sentence: Macronutrients Nutrients that are needed in large amounts are called macronutrients. There are three classes of macronutrients: carbohydrates, lipids, and proteins. These can be metabolically processed into cellular energy. The energy from macronutrients comes from their chemical bonds. This chemical energy is converted into cellular energy that is then utilized to perform work, allowing our bodies to conduct their basic functions. A unit of measurement of food energy is the calorie. On nutrition food labels the amount given for “calories” is actually equivalent to each calorie multiplied by one thousand. A kilocalorie (one thousand calories, denoted with a small “c”) is synonymous with the “Calorie” (with a capital “C”) on nutrition food labels. Water is also a macronutrient in the sense that you require a large amount of it, but unlike the other macronutrients, it does not yield calories. Carbohydrates Carbohydrates are molecules composed of carbon, hydrogen,

tensor([  42, 1290,   56,   47,  451])